In [1]:
import torch
import math
from torch.nn.functional import scaled_dot_product_attention

def gqa_scaled_dot_product_attention(query, key, value, n_groups: int) -> torch.Tensor:
    L, S = query.size(-2), key.size(-2)
    print(key.shape)
    key = torch.repeat_interleave(key, n_groups, dim=-3)
    print(key.shape)
    attn_weight = query @ key.transpose(-2, -1) 
    attn_weight = torch.softmax(attn_weight, dim=-1)
    value = torch.repeat_interleave(value, n_groups, dim=-3)
    return attn_weight @ value

In [2]:
# Testing the sdpa for 5d tensor (batch size, head, group, seq, seq)

In [3]:
q = torch.arange(400).view(1,4,1,10,10) * 1.0
kv = torch.arange(100).view(1,1,1,10,10) * 1.0

In [4]:
gqa_ = gqa_scaled_dot_product_attention(q, kv, kv, 4)
print(gqa_.shape)

torch.Size([1, 1, 1, 10, 10])
torch.Size([1, 1, 4, 10, 10])
torch.Size([1, 4, 4, 10, 10])


In [5]:
sdpa_ = scaled_dot_product_attention(query=q, key=kv, value=kv)
print(sdpa_.shape)

torch.Size([1, 4, 1, 10, 10])


In [6]:
# Testing the sdpa theory for 4d tensor (batch size, head, seq, seq)

In [7]:
q_ = torch.arange(3200).view(1,32,10,10) * 1.0
kv_ = torch.arange(800).view(1,8,10,10) * 1.0

gqa = gqa_scaled_dot_product_attention(q_, kv_, kv_, 4)
print(gqa.shape)

torch.Size([1, 8, 10, 10])
torch.Size([1, 32, 10, 10])
torch.Size([1, 32, 10, 10])


In [8]:
scaled_dot_product_attention(query=q_, key=kv_, value=kv_)
# This won't work coz first non-singleton dimension of q and kv needs to be same, hence for gqa we need to feed same size tensor 

RuntimeError: The size of tensor a (32) must match the size of tensor b (8) at non-singleton dimension 1

In [ ]:
kv_ = torch.repeat_interleave(kv_, 4, dim=-3)
sdpa = scaled_dot_product_attention(query=q_, key=kv_, value=kv_)

In [ ]:
torch.all(gqa == sdpa_)

In [ ]:
# sdpa_ = scaled_dot_product_attention(query=q, key=kv, value=kv)
# print(sdpa_.shape)